In [ ]:
!pip install pyspark
!pip install XlsxWriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# We must put into functions_v1.py
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import gspread
from google.auth import default
from gspread.models import Worksheet ,Cell
import xlsxwriter
from pyspark.sql import SparkSession, Row
from pyspark.sql.functions import row_number, monotonically_increasing_id, desc, concat_ws
from pyspark.sql.functions import col, when, lit, regexp_replace, from_json, max, expr, first
from pyspark.sql.window import Window
from pyspark.sql.dataframe import DataFrame
from pyspark.sql.types import *
from typing import List
from google.oauth2.sts import Client
import pyspark.sql.functions as F
from xlsxwriter.utility import *
from datetime import date
from datetime import datetime
from datetime import timedelta 
import functools
from functools import reduce
from pyspark.sql.functions import when
from xlsxwriter.utility import xl_col_to_name
from google.auth.impersonated_credentials import Credentials

In [ ]:
!cp "/functions_v1.py" /content

In [ ]:
from functions_v1 import create_df, map_sheet, create_dataframe, filter_controm_df, filter_batch_df, join_df

In [ ]:
#CHANGE EVERY MONTH
batch_report: str = '16jNcUeIjHzxxUF5EbUCAT3Z3egOZZRKKMd7qxWkN6dE'
sheet_month: str = 'Noviembre 2022'


sheet_executive_report: str = 'Informe Ejecutivo'

In [ ]:
def create_gspread():
  auth.authenticate_user()
  gauth: GoogleAuth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive: GoogleDrive = GoogleDrive(gauth)
  creds, _ = default()
  return gspread.authorize(creds)

In [ ]:
def run_batch_dataFrame(batch_id: str, spark: SparkSession, gc:Client) -> DataFrame :
  worksheet_batch: Worksheet = gc.open_by_key(batch_id).worksheet(sheet_month)
  batch_df: DataFrame = create_dataframe(worksheet_batch,spark)
  return batch_df.withColumn('index', row_number().over(Window.orderBy(monotonically_increasing_id())))\
  .filter(batch_df["Desplanificado"]==False), worksheet_batch #.select("Job en Control-M","Job en Argos - Dataproc","index")

In [ ]:
def run_executive_report_dataFrame(executive_report_id: str, spark: SparkSession, gc:Client) -> DataFrame :
  worksheet_executive_report: Worksheet = gc.open_by_key(executive_report_id).worksheet(sheet_executive_report)
  executive_report_df: DataFrame = create_dataframe(worksheet_executive_report,spark)
  executive_report_df = executive_report_df.filter(~executive_report_df["FECHA DIA"].isin("FECHA DIA"))
  # No podemos cambiar el nombre de la columna por '' de momento porque cambiaría todos los valores donde la columna es ''
  # La idea es crear dos bucles for, uno que cree la primera columna nula tipo Integer y otra que cree el tipo String
  new_executive_report_df = executive_report_df.withColumn('new_column1', lit(None).cast(StringType()))\
                                               .withColumn('new_column2', lit(None).cast(StringType()))\
                                               .withColumn('new_column3', lit(None).cast(StringType()))\
                                               .withColumn('new_column4', lit(None).cast(StringType()))\
                                               .withColumn('new_column5', lit(None).cast(StringType()))\
                                               .withColumn('new_column6', lit(None).cast(StringType()))\
                                               .withColumn('new_column7', lit(None).cast(StringType()))
                                               
  return new_executive_report_df, worksheet_executive_report

In [ ]:
def map_df_in_executive_report_worksheet(executive_report_df: DataFrame, worksheet_executive_report_batch: Worksheet, colu: List) -> None:
  try:    
    colum: str = xlsxwriter.utility.xl_col_to_name(worksheet_executive_report_batch.get_all_values()[0].index(colu))
    listIndex: List[tuple] = executive_report_df.select("new_index",str(colu)).where(col(colu).isNotNull()).rdd.map(tuple).collect()
    range: str = colum+"2:"+colum+"10000"
    cell_list: list[Cell] = worksheet_executive_report_batch.range(range)
    print(cell_list)
    i=0
    for cell in (cell for cell in cell_list if cell.row in list(map(lambda x:x[0],listIndex))):
      cell.value = listIndex[i][1]
      i=i+1
    worksheet_executive_report_batch.update_cells(cell_list)
  except:
    print(colum+" no esta en esta pagina")

In [ ]:
def map_executive_report_in_sheet(column_list: list, df:DataFrame, worksheet_to_map: Worksheet) -> None :
  for column_l in column_list:
    map_df_in_executive_report_worksheet(df,worksheet_to_map,column_l)

In [ ]:
def run_executive_report(batch_report:str) -> None :
  gc: Client = create_gspread()
  spark: SparkSession = SparkSession.builder.getOrCreate()
  batch_df , batch_sheet = run_batch_dataFrame(batch_report,spark,gc)
  executive_report_df, executive_report_sheet = run_executive_report_dataFrame(batch_report,spark,gc)
  worksheet_executive_report: Worksheet = gc.open_by_key(batch_report).worksheet(sheet_executive_report)
  columns_executive_report: list = executive_report_df.columns
  map_df_in_executive_report_worksheet(executive_report_df,worksheet_executive_report,columns_executive_report) 
  executive_report_df.show(5,False)

In [ ]:
run_executive_report(batch_report)

UnboundLocalError: ignored

In [ ]:
  gc: Client = create_gspread()
  spark: SparkSession = SparkSession.builder.getOrCreate()
  batch_df , batch_sheet = run_batch_dataFrame(batch_report,spark,gc)
  executive_report_df, executive_report_sheet = run_executive_report_dataFrame(batch_report,spark,gc)
  worksheet_executive_report: Worksheet = gc.open_by_key(batch_report).worksheet(sheet_executive_report)
  columns_executive_report: list = executive_report_df.columns

In [ ]:
type(columns_executive_report)

In [ ]:
executive_report_df.show(5)

+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|           FECHA DIA|         31/10/2022|         01/11/2022|         02/11/2022|         03/11/2022|         04/11/2022|         05/11/2022|         06/11/2022|         07/11/2022|new_column1|new_column2|new_column3|new_column4|new_column5|new_column6|new_column7|
+--------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+
|               ODATE|           20221030|           20221031|           20221101|           20221102|           20221103|           20221104|           20221105|           20221106|       null|     

In [ ]:
batch_df[batch_df.Proyecto == "CDD"].show(5)

+--------+---+-------------+--------------------+-----------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------+--------------+-------------+------+---------+--------------------+--------------------+-----------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+-----+
|Proyecto|   |Planificación|    Job en Control-M|Job en Argos - Dataproc|        Nombre Tabla|          Ruta Input|         Ruta Output|               Notas|            Reclamar|Acción|Desplanificado|Sin traspasar|Solape|Último KO|     Fichero Entrada|               Malla|Dependencia|20221101|20221102|20221103|20221104|20221105|20221106|20221107|20221108|20221109|20221110|20221111|20221112|20221113|20221114|20221115|20221116|